## Deleting columns

In [ ]:
# Import modules

import json
from http import HTTPStatus

import requests

## Prepare OAuth client and Evo configuration

In [ ]:
# Enter your access token, Evo hub URL, org ID, workspace ID, and block model ID.

access_token = ""
evo_hub_url = ""
org_id = ""
workspace_id = ""
block_model_id = ""

auth_header = {"Authorization": "Bearer " + access_token}

In [ ]:
# Compose the URL and JSON body
url = f"{evo_hub_url}/blockmodel/orgs/{org_id}/workspaces/{workspace_id}/block-models/{block_model_id}/blocks"

# The column that we are deleting was changed from "Geology" to "A new title!" in the previous notebook.
body = {
    "columns": {"new": [], "delete": ["A new title!"], "update": [], "rename": []},
    "comment": "Deleting the 'A new title!' column.",
}

# Make a PATCH request and include the JSON body and headers
response = requests.patch(url=url, json=body, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)

# Copy the `job_url` parameter from the response
if response.status_code == HTTPStatus.ACCEPTED:
    print(response_output)
    job_url = response.json()["job_url"]
else:
    raise Exception(f"Request failed: \n Status: {response.status_code} \n Response: {response_output}")

### Step 2: Tell BlockSync to process the change

After the `PATCH` request is complete, send a `POST` request to inform BlockSync that the change can now be processed.

The URL is simply the `job_url` from Step 1 with `/uploaded` appended.

NOTE: Even though no data was uploaded, this API request must still be made.

In [ ]:
# Compose the URL
url = f"{job_url}/uploaded"

# Make a POST request, including the headers
response = requests.post(url=url, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)
print(response_output)

### Step 3: Check the job status

Deleting data may take anywhere from a few seconds to a few minutes to complete, so it's still worthwhile checking the job status to ensure nothing went wrong.

In [ ]:
# Keep checking the `job_url` until the `job_status` is `COMPLETE`.

# Make a GET request, including the headers
response = requests.get(url=job_url, headers=auth_header)
response_output = json.dumps(response.json(), indent=4, sort_keys=True)
print(response_output)